<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/TF_kogpt2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers

In [19]:
import tensorflow as tf
from transformers import AutoTokenizer, GPT2Config, AddedToken
from transformers import TFGPT2LMHeadModel

모델 호출

In [36]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Add special tokens, including [SEP]
special_tokens = {'bos_token': '</s>', 'eos_token': '</s>', 'pad_token': '<pad>', 'sep_token': '<sep>', 'unk_token' : '<unk>'}
tokenizer.add_special_tokens(special_tokens)

# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')

# Add the new special token to the model's config

model_config.sep_token_id = tokenizer.sep_token_id  # Add this line


# Initialize the model
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', config=model_config, from_pt = True)

model.resize_token_embeddings(len(tokenizer))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

데이터셋 구축

In [39]:
import pandas as pd
import tqdm

In [40]:
# Initialize empty lists to store data
sentence1_list = []
sentence3_list = []
output_list = []

# Read the JSONL file line by line and extract data
with open("/content/drive/MyDrive/nikluge-sc-2023-train.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        output = data['output']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)
        output_list.append(output)

with open("/content/drive/MyDrive/nikluge-sc-2023-dev.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        output = data['output']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)
        output_list.append(output)

# Create a Pandas DataFrame
df = pd.DataFrame({'sentence1': sentence1_list, 'sentence3': sentence3_list, 'output': output_list})

In [41]:
train_data = df

In [49]:
batch_size = 20

In [50]:
def get_story_data():
  for sentence1, sentence3, output in zip(train_data.sentence1.to_list(), train_data.sentence3.to_list(), train_data.output.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<sys>' + sentence1 +'<sep>' + sentence3 + '<usr>' + output)
    yield bos_token + sent + eos_token


In [53]:
train_dataset = tf.data.Dataset.from_generator(get_story_data, output_types=tf.int32)

def pad_sequence_left(sequence, max_length):
    pad_length = max_length - tf.shape(sequence)[0]
    padded_sequence = tf.pad(sequence, [[pad_length, 0]], constant_values=tokenizer.pad_token_id)
    return padded_sequence

max_sequence_length = train_dataset.map(lambda x: tf.shape(x)).reduce(tf.constant([-1, -1], dtype=tf.int32), tf.maximum)[1]

train_dataset = train_dataset.map(lambda x: pad_sequence_left(x, max_sequence_length))
train_dataset = train_dataset.batch(batch_size)

In [55]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
steps = len(train_data) // batch_size + 1
print(steps)

6758


In [57]:
@tf.function
def training_step(batch):
    with tf.GradientTape() as tape:
        result = model(batch, labels=batch)
        loss = result[0]
        batch_loss = tf.reduce_mean(loss)

    grads = tape.gradient(batch_loss, model.trainable_variables)
    adam.apply_gradients(zip(grads, model.trainable_variables))
    return batch_loss


In [60]:
EPOCHS = 5

for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.tqdm_notebook(train_dataset, total=steps):
        batch_loss = training_step(batch)
        epoch_loss += batch_loss / steps

    print('[Epoch: {:>6}] cost = {:>.9}'.format(epoch + 1, epoch_loss))


<ipython-input-60-de4d4422e52c>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(train_dataset, total=steps):


  0%|          | 0/6758 [00:00<?, ?it/s]

[Epoch:      1] cost = 1.42275441


  0%|          | 0/6758 [00:00<?, ?it/s]

[Epoch:      2] cost = 1.25930786


...

[Epoch:      3] cost = 1.11181819


  0%|          | 0/6758 [00:00<?, ?it/s]

[Epoch:      4] cost = 0.96471709


  0%|          | 0/6758 [00:00<?, ?it/s]

[Epoch:      5] cost = 0.824744642


모델 저장

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [61]:
output_dir="Tiabet-tensorflow-finetuned-koGPT-complete_story"
model.push_to_hub(output_dir)

tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]